In [1]:
import warnings
warnings.simplefilter('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np

import tensorflow as tf
# Import the keras library
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.python.client import device_lib

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
print(os.environ['CUDA_VISIBLE_DEVICES'])

-1


In [4]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [5]:
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],X_train.shape[2],1))
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],X_test.shape[2],1))

y_train = y_train.reshape((y_train.shape[0],1))
y_test = y_test.reshape((y_test.shape[0],1))

In [7]:
%%timeit -n1 -r1

with tf.device('/CPU:0'):
    model_cpu = Sequential()
    model_cpu.add(Conv2D(input_shape = (28, 28, 1),
                     filters=5,
                     padding='Same',
                     kernel_size=(3,3)
                     ))
    model_cpu.add(MaxPooling2D(pool_size=(2,2)))
    model_cpu.add(Flatten())
    model_cpu.add(Dense(256, activation='relu'))
    model_cpu.add(Dense(10, activation='softmax'))

    model_cpu.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    model_cpu.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

Epoch 1/5
1875/1875 [==============================] - 30s 15ms/step - loss: 0.9320 - accuracy: 0.9164 - val_loss: 0.2017 - val_accuracy: 0.9503
Epoch 2/5
1875/1875 [==============================] - 28s 15ms/step - loss: 0.1391 - accuracy: 0.9637 - val_loss: 0.1677 - val_accuracy: 0.9578
Epoch 3/5
1875/1875 [==============================] - 29s 16ms/step - loss: 0.1036 - accuracy: 0.9717 - val_loss: 0.1611 - val_accuracy: 0.9655
Epoch 4/5
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0879 - accuracy: 0.9763 - val_loss: 0.1718 - val_accuracy: 0.9623
Epoch 5/5
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0702 - accuracy: 0.9811 - val_loss: 0.1494 - val_accuracy: 0.9686
2min 24s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Using GPU


In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [9]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7347065159186030867
xla_global_id: -1
]


In [10]:
%%timeit -n1 -r1
with tf.device('/device:GPU:2'):
    model_gpu = Sequential()
    model_gpu.add(Conv2D(input_shape = (28, 28, 1),
                     filters=5,
                     padding='Same',
                     kernel_size=(3,3)
                     ))
    model_gpu.add(MaxPooling2D(pool_size=(2,2)))
    model_gpu.add(Flatten())
    model_gpu.add(Dense(256, activation='relu'))
    model_gpu.add(Dense(10, activation='softmax'))

    model_gpu.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    model_gpu.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

Epoch 1/5
1875/1875 [==============================] - 40s 21ms/step - loss: 0.8807 - accuracy: 0.9117 - val_loss: 0.1748 - val_accuracy: 0.9503
Epoch 2/5
1875/1875 [==============================] - 27s 14ms/step - loss: 0.1218 - accuracy: 0.9657 - val_loss: 0.1553 - val_accuracy: 0.9577
Epoch 3/5
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0917 - accuracy: 0.9731 - val_loss: 0.1554 - val_accuracy: 0.9620
Epoch 4/5
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0817 - accuracy: 0.9757 - val_loss: 0.1541 - val_accuracy: 0.9636
Epoch 5/5
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0681 - accuracy: 0.9803 - val_loss: 0.1453 - val_accuracy: 0.9692
3min 22s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [11]:
tf.debugging.set_log_device_placement(True)

gpus = tf.config.list_logical_devices('GPU')
if gpus:
    c = []
    for gpu in gpus:
        with tf.device(gpu.name):
            a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
            b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
            c.append(tf.matmul(a, b))

    # Run on CPU
    with tf.device('/CPU:0'):
        matmul_sum = tf.add_n(c)

    print(matmul_sum)